Demo of a single and 2 layer perceptron using numpy and pysyft

Based on http://iamtrask.github.io/2015/07/12/basic-python-network/

In [1]:
import numpy as np
from syft.syft import FloatTensor,SyftController

Using numpy

In [2]:
#4 inupts samples
x = np.array([ [0,0,1],[1,1,1],[1,0,1],[0,1,1] ])#works
#x = np.array([ [0,0,1],[0,1,1],[1,0,1],[1,1,1] ])#xor gate need hidden layer
y = np.array([[0,1,1,0]]).T
#perceprton weights
np.random.seed(0)
w_initial = 2*np.random.random((len(x[0]),len(y[0]))) - 1
#activation function
def sigmoid(x,deriv=False):
    if(deriv==True):
        return x*(1-x)#the derivative is sigmoid(sigmoid(x),1)
    return 1/(1+np.exp(-x))
#predicted y pre training
print(sigmoid(np.dot(x,w_initial)))
#training
w = w_initial.copy()
#for j in range(60000):
for j in range(1):
    l1 = sigmoid(np.dot(x,w))
    l1_delta = (y - l1)*sigmoid(l1,1)
    w += x.T.dot(l1_delta)
    if j%10000 == 0:
        print("Error: ",np.average(np.abs((y - l1))))
#predicted y post training
print( l1 )

[[ 0.55120158]
 [ 0.67557997]
 [ 0.5752133 ]
 [ 0.6538273 ]]
Error:  0.48855890289
[[ 0.55120158]
 [ 0.67557997]
 [ 0.5752133 ]
 [ 0.6538273 ]]


Using pysyft

In [3]:
# connect to unity
identity = 'worker-0'
sc = SyftController(identity)

put data into openmined

In [4]:
X = sc.FloatTensor( x )
Y = sc.FloatTensor( y )
W_initial = sc.FloatTensor( w_initial )
W = sc.FloatTensor( w_initial )
zeros = sc.FloatTensor(np.array([[0,0,0,0]]).T)#needed as we don't have mm yet
ones = sc.FloatTensor(np.array([[1,1,1,1]]).T)#needed as we don't deriv of sigmoid yet
mean = sc.FloatTensor(np.array([[1/4,1/4,1/4,1/4]]))#hack to calculate mean
zero = sc.FloatTensor(np.array([[0]]))#also for mean

FloatTensor.__init__: 1
FloatTensor.__init__: 2
FloatTensor.__init__: 3
FloatTensor.__init__: 4
FloatTensor.__init__: 5
FloatTensor.__init__: 6
FloatTensor.__init__: 7
FloatTensor.__init__: 8


In [5]:
#X.gpu()
#Y.gpu()
#W.gpu()

In [6]:
#predicted pre training
predition=zeros.addmm(X,W)
predition.sigmoid_()
print(predition)
#training
for j in range(1):
    L1 = zeros.addmm(X,W)
    L1.sigmoid_()
    L1_delta = (Y - L1)*(L1*(ones-L1))
    W.addmm_(X.T(),L1_delta)
    if j%1 == 0:
        err = (Y - L1)
        err.abs_()
        print("Error: ",zero.addmm(mean,err))
#predicted y post training
print( l1 )

FloatTensor.__init__: 9
0.5512016,	
0.67558,	
0.5752133,	
0.6538273,	


FloatTensor.__init__: 10
FloatTensor.__init__: 15
FloatTensor.__init__: 17
Error:  0.4885589,	


[[ 0.55120158]
 [ 0.67557997]
 [ 0.5752133 ]
 [ 0.6538273 ]]


In [7]:
w_initial,W_initial

(array([[ 0.09762701],
        [ 0.43037873],
        [ 0.20552675]]), 0.09762701,	
 0.4303787,	
 0.2055268,	
 )

In [8]:
l1,L1

(array([[ 0.55120158],
        [ 0.67557997],
        [ 0.5752133 ],
        [ 0.6538273 ]]), 0.5512016,	
 0.67558,	
 0.5752133,	
 0.6538273,	
 )

In [9]:
l1_delta,L1_delta

(array([[-0.13635536],
        [ 0.07110368],
        [ 0.10379364],
        [-0.14798542]]), -0.1363554,	
 0.07110368,	
 0.1037936,	
 -0.1479854,	
 )

In [10]:
x,X

(array([[0, 0, 1],
        [1, 1, 1],
        [1, 0, 1],
        [0, 1, 1]]), 0,	0,	1,	
 1,	1,	1,	
 1,	0,	1,	
 0,	1,	1,	
 )

Above all ok, but below should be the same also. Why is below not the same?
Code is 
    w += x.T.dot(l1_delta)
    W.addmm_(X.T(),L1_delta)

In [11]:
w,W

(array([[ 0.27252433],
        [ 0.353497  ],
        [ 0.09608329]]), 0.05343522,	
 0.4689207,	
 0.02497962,	
 )